In [1]:
from ibl_pipeline import behavior as behavior_analyses
from ibl_pipeline import behavior as behavior
from ibl_pipeline import subject, acquisition, action
from ibl_pipeline.utils import psychofit as psy
import plotting_utils as putils
import numpy as np
import datetime
import datajoint as dj
import json
import pandas as pd
from uuid import UUID

Connecting shan@datajoint.internationalbrainlab.org:3306


In [2]:
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

In [3]:
water_type_names = action.WaterType.fetch('watertype_name')
water_type_colors = ['red', 'orange', 'blue', 'rgba(55, 128, 191, 0.7)', 'purple', 'rgba(50, 171, 96, 0.9)'] 
water_type_map = dict()
# get date range
d = putils.get_date_range(subj)

for watertype, color in zip(water_type_names, water_type_colors):
    water_type_map.update({watertype: color})

In [4]:
subj = subject.Subject & {'subject_nickname': 'ibl_witten_10'}
# get water and date
water_info_query = (action.WaterAdministration & subj).proj(
    'water_administered', 'watertype_name', water_date='DATE(administration_time)')
water_info = water_info_query.fetch(as_dict=True)
water_info = pd.DataFrame(water_info)
water_types = water_info.watertype_name.unique()
water_info_type = water_info.pivot_table(
    index='water_date', columns='watertype_name', values='water_administered', aggfunc='sum')
max_water_intake = np.nanmax(water_info_type.values) + 0.2
yrange_water = [0, max_water_intake]
water_info_type = water_info_type.where((pd.notnull(water_info_type)), None)
weight_info_query = (action.Weighing & subj).proj(
    'weight', weighing_date='DATE(weighing_time)')
weight_info = weight_info_query.fetch(as_dict=True)
weight_info = pd.DataFrame(weight_info)
weight_info = weight_info.where((pd.notnull(weight_info)), None)

# get water restriction period
water_restrictions = (action.WaterRestriction & subj).proj(
    'reference_weight',
    res_start='DATE(restriction_start_time)', res_end='DATE(restriction_end_time)')


data = [
    go.Bar(
        x = [t.strftime('%Y-%m-%d') for t in water_info_type.index.tolist()],
        y = water_info_type[water_type].tolist(),
        marker = dict(color = water_type_map[water_type]),
        name = water_type,
        yaxis = 'y1',
        legendgroup='water_type'
    )
    for water_type in water_types
]

data.append(
    go.Scatter(
        x=[t.strftime('%Y-%m-%d') for t in weight_info['weighing_date'].tolist()],
        y=weight_info['weight'].tolist(),
        mode='lines+markers',
        name='Weight',
        marker=dict(
            size=6,
            color='black',
            line = dict(
            color = 'white',
            width = 1)
        ),
        legendgroup='weight',
        yaxis = 'y2'
    ))

# monday marks
data = putils.create_monday_plot(data, yrange_water, d['mondays'])

# water restriction marks and reference weight marks
for iwater, water_res in enumerate(water_restrictions.fetch(as_dict=True)):
    
    if iwater==0:
        show_res_legend = True
    else:
        show_res_legend = False
    
    res_start = water_res['res_start'].strftime('%Y-%m-%d')
    
    if water_res['res_end']:
        res_end = water_res['res_end'].strftime('%Y-%m-%d')
    else:
        res_end = last_date_str
    data.append(
        go.Scatter(
            x=[res_start, res_start],
            y=yrange_water,
            mode="lines",
            line=dict(
                width=1,
                color='red',
            ),
            name='Water restriction start',
            yaxis='y1',
            showlegend=show_res_legend,
            legendgroup='restriction'
        )
    )
    
    if water_res['res_end']:

        data.append(
            go.Scatter(
                x=[res_end, res_end],
                y=yrange_water,
                mode="lines",
                line=dict(
                    width=1,
                    color='darkgreen',
                ),
                name='Water restriction end',
                yaxis='y1',
                showlegend=show_res_legend,
                legendgroup='restriction'
            )
        )
    
     
    data.append(
        go.Scatter(
            x=[res_start, res_end],
            y=[water_res['reference_weight']*0.85, water_res['reference_weight']*0.85],
            mode="lines",
            line=dict(
                width=1,
                color='orange',
                dash='dashdot'
            ),
            name='85% reference weight',
            yaxis='y2',
            showlegend=show_res_legend,
            legendgroup='weight_ref'
        )
    )
    
    data.append(
        go.Scatter(
            x=[res_start, res_end],
            y=[water_res['reference_weight']*0.75, water_res['reference_weight']*0.75],
            mode="lines",
            line=dict(
                width=1,
                color='red',
                dash='dashdot'
            ),
            name='75% reference weight',
            yaxis='y2',
            showlegend=show_res_legend,
            legendgroup='weight_ref'
        )
    )



layout = go.Layout(
    yaxis=dict(
        title='Water intake (mL)',
        range=yrange_water
    ),
    yaxis2=dict(
        title='Weight (g)',
        overlaying='y',
        side='right',
    ),
    width=1000,
    height=500,
    title=dict(
        text='Water intake and weight',
        x=0.3,
        y=0.9
    ),
    xaxis=dict(
        title='Date',
        range=[d['first_date_str'], d['last_date_str']]
    ), 
    legend=dict(
        x=1.1, 
        y=0.9,
        orientation='v'),
    barmode='stack'
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [6]:
f = open("water_weight.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()